In [2]:
import baostock as bs
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

# -------------------------------
# 目标设置
# -------------------------------
code = "sh.601328"           # Baostock 股票代码
freq = "5"                  # 5 分钟
start_date = "2019-01-01"   # 五年前起始日期
end_date = pd.to_datetime("today").strftime("%Y-%m-%d")  # 当前日期

output_csv = "601328_5min_last5years.csv"

# -------------------------------
# 登录 Baostock
# -------------------------------
lg = bs.login()
print("登录状态:", lg.error_code, lg.error_msg)

# -------------------------------
# 拉取函数
# -------------------------------
def fetch_baostock_5min(code, start_date, end_date):
    all_data = []
    cur = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)

    # 生成日期列表用于进度
    date_list = pd.date_range(start=start_date, end=end_date, freq="D")

    for dt in tqdm(date_list, desc="拉取日期进度"):
        day_str = dt.strftime("%Y-%m-%d")

        rs = bs.query_history_k_data_plus(
            code,
            "date,time,open,high,low,close,volume",
            start_date=day_str,
            end_date=day_str,
            frequency=freq,
            adjustflag="3"
        )

        # 读取数据
        data_list = []
        while rs.error_code == "0" and rs.next():
            data_list.append(rs.get_row_data())

        if data_list:
            df_day = pd.DataFrame(data_list, columns=rs.fields)
            all_data.append(df_day)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

# -------------------------------
# 执行拉取
# -------------------------------
print(f"开始从 {start_date} 到 {end_date} 拉取 {code} 的 5 分钟数据...")
df_5min = fetch_baostock_5min(code, start_date, end_date)

# -------------------------------
# 保存 CSV
# -------------------------------
if not df_5min.empty:
    df_5min.to_csv(output_csv, index=False)
    print("保存成功:", output_csv)
    print("数据样例:\n", df_5min.head())
    print("总行数:", len(df_5min))
else:
    print("没有拉取到任何数据！")

bs.logout()


login success!
登录状态: 0 success
开始从 2019-01-01 到 2026-01-11 拉取 sh.601328 的 5 分钟数据...


拉取日期进度: 100%|██████████| 2568/2568 [02:04<00:00, 20.63it/s]


保存成功: 601328_5min_last5years.csv
数据样例:
          date               time    open    high     low   close   volume
0  2019-01-02  20190102093500000  5.7700  5.8000  5.7600  5.7700  2513490
1  2019-01-02  20190102094000000  5.7700  5.7800  5.7500  5.7500  1420900
2  2019-01-02  20190102094500000  5.7600  5.7600  5.7300  5.7300  1725200
3  2019-01-02  20190102095000000  5.7300  5.7300  5.7000  5.7000  4243200
4  2019-01-02  20190102095500000  5.7100  5.7100  5.6800  5.6800  2477100
总行数: 81792
logout success!
